# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [2]:
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 1.6 MB/s eta 0:00:00a 0:00:01


In [7]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [4]:
# Load API Credentials
with open('/Users/hkim1297/.secret/yelp_api.json') as f:
    creds = json.load(f)
creds.keys()

dict_keys(['client-id', 'api-key'])

In [9]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(creds['api-key'])
yelp

### Define Search Terms and File Paths

In [10]:
# set our API call parameters and filename before the first call
LOCATION = "Seattle, WA 98122"
TERM = "Coffee"

In [11]:
## Specify folder for saving data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+"Seattle Coffee.json"

In [12]:
LOCATION.split(',')[0]

'Seattle'

In [22]:
# ANOTHER way of # Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{LOCATION.split(',')[0]} - {TERM}.json"

### Check if Json File exists and Create it if it doesn't

In [23]:
## Check if JSON_FILE exists
os.path.isfile(JSON_FILE)

False

In [24]:

## If it does not exist: 
    print('The file does not exist. Creating empty file')
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    with open(JSON_FILE, 'w') as f:
        json.dump(results['businesses'], f)
    
    ## If JSON_FILE included a folder:

        # create the folder

        
        
    ## INFORM USER AND SAVE EMPTY LIST

    
    
    ## save the first page of results

        
## If it exists, inform user
else:
    print('it already exists')

The file does not exist. Creating empty file


### Load JSON FIle and account for previous results

In [25]:
## Load previous results and use len of results for offset
prev = pd.read_json(JSON_FILE)
## set offset based on previous results
nResults = len(prev) - 700
nResults

20

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [13]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term=TERM, location=LOCATION)
type(results)

dict

In [15]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
len(results['businesses']) # only from first page

20

In [19]:
## How many results total?
results['total']

1500

In [20]:
# let's save as a df
df = pd.DataFrame(results['businesses'])
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,9TnUll8eOc5FREIQqrcrFg,verite-coffee-seattle,Verite Coffee,https://s3-media4.fl.yelpcdn.com/bphoto/SaSFeE...,False,https://www.yelp.com/biz/verite-coffee-seattle...,39,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,"{'latitude': 47.61175, 'longitude': -122.28953}",[],$,"{'address1': '1101 34th Ave', 'address2': '', ...",+12067094497,(206) 709-4497,483.915129
1,yLZborRy-JRAKtTWBk9a9g,union-coffee-and-wine-seattle,Union Coffee and Wine,https://s3-media2.fl.yelpcdn.com/bphoto/99K9k7...,False,https://www.yelp.com/biz/union-coffee-and-wine...,94,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.61274, 'longitude': -122.30103}",[delivery],$,"{'address1': '2407 E Union St', 'address2': 'S...",+17066314474,(706) 631-4474,406.415695
2,mUtrinuyznLic5wYc09LeA,arosa-cafe-seattle-6,Arosa Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Sz1XcR...,False,https://www.yelp.com/biz/arosa-cafe-seattle-6?...,103,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.62620255, 'longitude': -122.29...","[delivery, pickup]",$,"{'address1': '3121 E Madison St', 'address2': ...",+12064050605,(206) 405-0605,1490.203135
3,yCLUXtS6a1xMicPSsmdTfA,squirrel-chops-seattle,Squirrel Chops,https://s3-media2.fl.yelpcdn.com/bphoto/0fA2eb...,False,https://www.yelp.com/biz/squirrel-chops-seattl...,89,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.6128151, 'longitude': -122.303...",[],$,"{'address1': '2201 E Union St', 'address2': ''...",+12064857017,(206) 485-7017,572.723877
4,Q8Nlx_FeZ4LW0kdbnHoyrw,qed-coffee-seattle,QED Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/HwagNI...,False,https://www.yelp.com/biz/qed-coffee-seattle?ad...,104,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.5896816513796, 'longitude': -1...",[delivery],$$,"{'address1': '1418 31st Ave S', 'address2': ''...",+12062357678,(206) 235-7678,2615.675293


- Where is the actual data we want to save?

In [26]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [27]:
# Use math.ceil to round up for the total number of pages of results.
import math
nPages = math.ceil(results['total']/results_per_page)
nPages

75

In [28]:
for i in tqdm_notebook( range(1,3)):
    ## The block of code we want to TRY to run
        
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE) as f:
            prevResults = json.load(f)
            
        ## set offset based on previous results
        nResults = len(prevResults)
        ## save number of results for to use as offset
        results = yelp.search_query(term=TERM, location=LOCATION, offset=nResults)
        
        
        ## use n_results as the OFFSET 
        results = 
        
        ## append new results and save to file
        
        prevResults.extend(results['businesses'])
        with open(JSON_FILE, 'w') as f:
            json.dump(prevResults,f)
            
    ## What to do if we get an error/exception.
        


SyntaxError: invalid syntax (2670142990.py, line 16)

## Open the Final JSON File with Pandas

In [ ]:
df = None

In [ ]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

In [ ]:
## Save it as a compressed csv (to save space)


## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

## Next Class: Processing the Results and Mapping 